# Replacing semicolon with comma 

In [7]:
with open('/home/hp/Desktop/Thesis/sortedData/test/xaa', "r+", encoding="utf-8") as csv_file: content = csv_file.read()

with open('/home/hp/Desktop/Thesis/sortedData/xaa', "w+", encoding="utf-8") as csv_file:
    csv_file.write(content.replace(';', ','))

# Getting Unique column/Table names from all the sql queries

In [48]:
import sql_metadata as sq
from sql_metadata import Parser



a=Parser("""SELECT COUNT(*) FROM cast_info ci WHERE ci.person_id=172968,
SELECT COUNT(*) FROM title t,movie_info mi WHERE t.id=mi.movie_id AND t.kind_id<3 AND t.production_year=2008 AND mi.info_type_id>2,
SELECT COUNT(*) FROM title t,cast_info ci WHERE t.id=ci.movie_id AND ci.person_id<3194645,
SELECT COUNT(*) FROM title t,cast_info ci,movie_info mi WHERE t.id=ci.movie_id AND t.id=mi.movie_id AND ci.person_id=1742124 AND ci.role_id>2 AND mi.info_type_id<7,
SELECT COUNT(*) FROM title t,cast_info ci,movie_info_idx mi_idx WHERE t.id=ci.movie_id AND t.id=mi_idx.movie_id AND t.kind_id=7 AND t.production_year>0 AND ci.role_id=2 AND mi_idx.info_type_id=101,
SELECT COUNT(*) FROM title t,movie_companies mc WHERE t.id=mc.movie_id AND t.production_year<2000 AND mc.company_id=160 AND mc.company_type_id=2,
SELECT COUNT(*) FROM title t,movie_companies mc,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mc.company_id<77942 AND mc.company_type_id>1,
SELECT COUNT(*) FROM movie_keyword mk WHERE mk.keyword_id<71692,
SELECT COUNT(*) FROM title t,cast_info ci,movie_info_idx mi_idx WHERE t.id=ci.movie_id AND t.id=mi_idx.movie_id AND ci.person_id>681261 AND ci.role_id>1 AND mi_idx.info_type_id=99,
SELECT COUNT(*) FROM movie_companies mc WHERE mc.company_id<9749 AND mc.company_type_id>1,
SELECT COUNT(*) FROM title t,movie_companies mc,movie_keyword mk WHERE t.id=mc.movie_id AND t.id=mk.movie_id AND mc.company_id=43268 AND mk.keyword_id<2560,
SELECT COUNT(*) FROM cast_info ci WHERE ci.person_id>1278953 AND ci.role_id<2,
SELECT COUNT(*) FROM title t,movie_keyword mk WHERE t.id=mk.movie_id AND mk.keyword_id<1029,
SELECT COUNT(*) FROM title t,movie_companies mc,movie_info mi WHERE t.id=mc.movie_id AND t.id=mi.movie_id AND t.kind_id<7 AND mc.company_id>83328 AND mi.info_type_id<8,
SELECT COUNT(*) FROM title t,movie_companies mc,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND t.kind_id=7 AND t.production_year=1994 AND mc.company_id>50785 AND mc.company_type_id>1,
SELECT COUNT(*) FROM movie_companies mc WHERE mc.company_id<80833 AND mc.company_type_id=1,
SELECT COUNT(*) FROM title t,cast_info ci WHERE t.id=ci.movie_id AND ci.person_id<2945474 AND ci.role_id=2,
SELECT COUNT(*) FROM movie_keyword mk WHERE mk.keyword_id>2237,
select rani.raja from rani""").columns

print(a)


['cast_info', 'title', 'movie_info', 'movie_info_idx', 'movie_companies', 'movie_keyword', 'rani']


# Extracting table name(not unique) from sql queries

In [45]:
f = open('/home/hp/Desktop/Thesis/sortedData/test.txt', "r")
b=f.read()

import itertools
import sqlparse

from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML


def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False


def extract_from_part(parsed):
    from_seen = False
    for item in parsed.tokens:
        if item.is_group:
            for x in extract_from_part(item):
                yield x
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
            elif item.ttype is Keyword and item.value.upper() in ['ORDER', 'GROUP', 'BY', 'HAVING', 'GROUP BY']:
                from_seen = False
                StopIteration
            else:
                yield item
        if item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True


def extract_table_identifiers(token_stream):
    for item in token_stream:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                value = identifier.value.replace('"', '').lower()                
                yield value
        elif isinstance(item, Identifier):
            value = item.value.replace('"', '').lower()
            yield value


def extract_tables(sql):
    # let's handle multiple statements in one sql string
    extracted_tables = []
    statements = list(sqlparse.parse(sql))
    for statement in statements:
        if statement.get_type() != 'UNKNOWN':
            stream = extract_from_part(statement)
            extracted_tables.append(set(list(extract_table_identifiers(stream))))
    return list(itertools.chain(*extracted_tables))


if __name__ == '__main__':
#     sql_str = """
#     SELECT COUNT(*) FROM cast_info ci WHERE ci.person_id=172968;
#     """

    tables = ', '.join(extract_tables(b))
    a=('Tables: {}'.format(tables))
    print(a)

Tables: cast_info ci, title t, movie_info mi, title t, cast_info ci, title t, movie_info mi, cast_info ci, movie_info_idx mi_idx, title t, cast_info ci, title t, movie_companies mc, movie_info_idx mi_idx, title t, movie_companies mc, movie_keyword mk, movie_info_idx mi_idx, title t, cast_info ci, movie_companies mc, title t, movie_companies mc, movie_keyword mk, cast_info ci, title t, movie_keyword mk, title t, movie_info mi, movie_companies mc, movie_info_idx mi_idx, title t, movie_companies mc, movie_companies mc, title t, cast_info ci, movie_keyword mk, title t, movie_companies mc, movie_info_idx mi_idx, title t, movie_companies mc, title t, movie_keyword mk, title t, movie_companies mc, title t, cast_info ci, title t, movie_keyword mk, movie_info_idx mi_idx, title t, movie_info mi, title t, movie_info mi, movie_companies mc, cast_info ci, title t, cast_info ci, title t, movie_companies mc, movie_companies mc, title t, movie_companies mc, movie_info_idx mi_idx, title t, movie_keywor

In [47]:
with open('/home/hp/Desktop/Thesis/sortedData/Table_name.csv', 'w') as f:
    for item in a:
        f.write( item)        

# Count frequency of words in the file

In [52]:
f = open('/home/hp/Desktop/Thesis/sortedData/Table_name.csv', "r")
b=f.read()
elm_count = b.count('movie_companies mc')
print('The count of column info_type_id : ', elm_count)

The count of column info_type_id :  1550
